In [3]:
import itc_utils.flight_service as itcfs

import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("fs.s3a.endpoint", "https://s3.eu-de.cloud-object-storage.appdomain.cloud") \
    .config("fs.s3a.access.key","4736b28acb064baeb7701a1f416d2b0c") \
    .config("fs.s3a.secret.key","1a070473a7fb93f4a13a1f5ab78bf2c82cd16649c9c4573d") \
    .getOrCreate()

#     .config("spark.jars", "delta-storage-2.1.1.jar, delta-core_2.12-2.1.1.jar") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \

# NOTE:
#  A limit of 5000 rows has been applied to the request to enable sample previewing.
#  Adjust the display message as needed by editing the following lines:
from IPython.display import display, HTML
display(HTML("A row limit of 1000 has been applied to the query to enable sample previewing. If the data set is larger, only the first 5000 rows will be loaded."))
#  Edit select_statement to change or disable the row limit.
#
Trial_DB2_data_request = {
    'connection_name': """Trial_DB2""",
    'interaction_properties': {
        'select_statement': 'SELECT * FROM "CUSTOMER"."CUSTOMER" FETCH FIRST 1000 ROWS ONLY'
    }
}
flight_request = itcfs.get_data_request(nb_data_request=Trial_DB2_data_request)

sp_df_2 = sparkSession.read.format("com.ibm.connect.spark.flight") \
    .option("flight.location", itcfs.get_flight_service_url()) \
    .option("flight.command", itcfs.get_flight_cmd(data_request=flight_request)) \
    .option("flight.authToken", itcfs.get_bearer_token()) \
    .load()
# sp_df_2.show(10)


RuntimeError: No asset named "Trial_DB2" of type "connection".

In [1]:
## Si queremos cargar un fichero csv en el proyecto, en vez de ir al Warehouse
# import itc_utils.flight_service as itcfs

# import pyspark.sql.functions as f
# from pyspark.sql.window import Window
# from pyspark.sql import SparkSession
# sparkSession = SparkSession.builder \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("fs.s3a.endpoint", "https://s3.eu-de.cloud-object-storage.appdomain.cloud") \
#     .config("fs.s3a.access.key","4736b28acb064baeb7701a1f416d2b0c") \
#     .config("fs.s3a.secret.key","1a070473a7fb93f4a13a1f5ab78bf2c82cd16649c9c4573d") \
#     .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
#     .getOrCreate()
# #spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# #     .config("spark.jars", "delta-storage-2.1.1.jar, delta-core_2.12-2.1.1.jar") \
# #     .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \

# readClient = itcfs.get_flight_client()

# nb_data_request = {
#     'data_name': """CUSTOMER_INCOME.csv""",
#     'interaction_properties': {
#         #'row_limit': 500,
#         'infer_schema': 'true',
#         'infer_as_varchar': 'false'
#     }
# }

# flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

# data_df_0 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
# sp_df_3 = spark.createDataFrame(data_df_0)
# # sp_df_3.show(10)


In [ ]:
## Si queremos cargar un fichero csv en el proyecto, en vez de ir al Warehouse
# import pandas as pd
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()

# customer_pandas = sp_df_2.toPandas()
# customer_pandas_csv = customer_pandas.to_csv(index=False)
# wslib.save_data("CUSTOMER_INCOME.csv", customer_pandas_csv.encode(), overwrite=True)

In [2]:
def filter_columns(df):
  return df.select("FIRST_NAME", "LAST_NAME", "GENDER", "COUNTRY", "INCOME").orderBy("INCOME", ascending = False)

In [3]:
customer_filtered = filter_columns(sp_df_2)

In [4]:
customer_filtered.write.format("delta").mode("overwrite").save("s3a://sparkdatabucket/customer_filtered/")

In [6]:
# load_cust_filt = sparkSession.read.format("delta").load("s3a://sparkdatabucket/customer_filtered/")

In [9]:
# load_cust_filt.show()

+----------+---------+------+--------------+------+
|FIRST_NAME|LAST_NAME|GENDER|       COUNTRY|INCOME|
+----------+---------+------+--------------+------+
|   Rolanda|     Ring|female|        Canada| 99960|
|   Marquis|    Jiron|  male|       Germany| 99845|
|   Nicolle|   Zollar|female|       Germany| 99824|
|   Chantal|  Kammler|female| United States| 99205|
|   Lorette|  Rochell|  male|       Germany| 98912|
|  Porfirio|    Crupi|  male| United States| 98701|
|       Kym|    Grime|  male|United Kingdom| 98132|
|    Collin|   Renyer|female|United Kingdom| 97768|
|     Bruna|     Gadd|  male| United States| 97745|
|    Leslee|  Chamers|  male| United States| 97460|
|     Steve|   Grippe|  male|        Canada| 97431|
|     Leila|    Sebek|female|        Canada| 97361|
|      Dean| Selgrade|  male| United States| 97298|
|     Tessa| Mintreas|female|United Kingdom| 97245|
|     Randy|   Bervig|female|       Germany| 97162|
|    Kelsie|   Turick|  male|       Germany| 97026|
|    Astrid|